In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
% matplotlib inline

In [3]:
df = pd.read_csv("all_attributes.csv")
targets = df.iloc[:, [8, 9, 10, 11, 12]]
independents = df.drop(df.columns[[0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 19]], axis=1)
independents.head()

,beer/ABV,beer/beerId,beer/brewerId,stopwordcount,charcount,cursewordcount,uniquecursewordcount,exceptional,excellent,fantastic,wonderful,highly,ok,average,bad,corn,sipper,group_lda
0,5.0,46634,14338,42,415,0,0,0,0,0,0,0,0,0,0,0,0,0
1,11.0,3003,395,42,450,0,0,0,0,0,0,0,0,0,1,0,0,1
2,4.7,961,365,21,312,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4.4,429,1,27,312,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.4,4904,1417,49,383,0,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
targets.head()

,review/appearance,review/aroma,review/overall,review/palate,review/taste
0,4.0,4.0,4.0,4.0,4.0
1,4.0,3.5,3.5,3.5,3.0
2,3.5,4.0,3.5,3.5,3.5
3,3.0,3.0,2.5,3.0,3.0
4,4.0,3.0,3.0,3.5,2.5


#### Apearance

In [10]:
X = independents
y = targets.iloc[:, 0]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
print("Lasso R^2: " + str(las_reg.score(X, y)))
print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.152332657664
Regular SSE: 11013.487388324798
Lasso R^2: 0.0662758334632
Lasso SSE: 12131.597878938586


#### Aroma

In [11]:
X = independents
y = targets.iloc[:, 1]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
print("Lasso R^2: " + str(las_reg.score(X, y)))
print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.237628626975
Regular SSE: 13246.411963577131
Lasso R^2: 0.0414001889066
Lasso SSE: 16655.934961937604


#### Palate

In [12]:
X = independents
y = targets.iloc[:, 3]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
print("Lasso R^2: " + str(las_reg.score(X, y)))
print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.203779391689
Regular SSE: 13319.336587065038
Lasso R^2: 0.0782111472506
Lasso SSE: 15419.86713709262


#### Taste

In [13]:
X = independents
y = targets.iloc[:, 4]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
print("Lasso R^2: " + str(las_reg.score(X, y)))
print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.235045616963
Regular SSE: 14721.54891751806
Lasso R^2: 0.0872391601305
Lasso SSE: 17566.08453015123


#### Overall

In [14]:
X = independents
y = targets.iloc[:, 2]

regr = linear_model.LinearRegression().fit(X, y)
print("Regular R^2: " + str(regr.score(X, y)))
print("Regular SSE: " + str(sum((regr.predict(X) - y) ** 2)))

las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
print("Lasso R^2: " + str(las_reg.score(X, y)))
print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Regular R^2: 0.135896454203
Regular SSE: 15892.185145868403
Lasso R^2: 0.0237631327677
Lasso SSE: 17954.48834313506
